In [2]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU , Bidirectional, Dropout
from keras.callbacks import ModelCheckpoint
from random import randint
import tensorflow as tf

In [4]:
# Read in the data csv file into one single string named corpus. 
corpus = open('abba.csv', 'r').read()
print(corpus)

,index,song,year,artist,genre,lyrics
130805,130805,waterloo-english-version,2009,abba,Pop,"My my
At Waterloo Napoleon did surrender
Oh yeah
And I have met my destiny in quite a similar way
The history book on the shelf
Is always repeating itself
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my Waterloo
My my
I tried to hold you back, but you were stronger
Oh yeah
And now it seems my only chance is giving up the fight
And how could I ever refuse
I feel like I win when I lose
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Oh, oh Waterloo finally facing my Waterloo
So how could I ever refuse
I feel like I win when I lose
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my

In [6]:
# Clean the data.
# Remove unicode characters, \r and other special characters.
# Save in file to check
corpus = corpus.replace('\r', '')
print(corpus)

# Save the cleaned data to a file to check
with open('cleaned_data.csv', 'w') as file:
    file.write(corpus)






,index,song,year,artist,genre,lyrics
130805,130805,waterloo-english-version,2009,abba,Pop,"My my
At Waterloo Napoleon did surrender
Oh yeah
And I have met my destiny in quite a similar way
The history book on the shelf
Is always repeating itself
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my Waterloo
My my
I tried to hold you back, but you were stronger
Oh yeah
And now it seems my only chance is giving up the fight
And how could I ever refuse
I feel like I win when I lose
Waterloo I was defeated, you won the war
Waterloo promise to love you for ever more
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Oh, oh Waterloo finally facing my Waterloo
So how could I ever refuse
I feel like I win when I lose
Waterloo couldn't escape if I wanted to
Waterloo knowing my fate is to be with you
Waterloo finally facing my

In [7]:
# Create encoder and decoder dictionaries.
# each character is mapped to an integer and vice versa.
chars = sorted(list(set(corpus)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}


In [8]:
# slice the corpus into semi-redundant sequences of 20 characters, and encode them using our dictionaries.
# it slices, it dices, it makes julienned datasets!
# chop up our data into X and y, slice into roughly (num_chars / skip) overlapping 'sentences'
# of length sentence_length, and encode the chars
# Mischien is sentence_length te kort na 20 een nieuw char voorspellen?.
sentence_length = 20

skip = 1
X_data = []
y_data = []

for i in range (0, len(corpus) - sentence_length, skip):
    sentence = corpus[i:i + sentence_length]
    next_char = corpus[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])




In [11]:
# simple check.
X_data[1], y_data[1]


([58, 63, 53, 54, 73, 7, 68, 64, 63, 56, 7, 74, 54, 50, 67, 7, 50, 67, 69, 58],
 68)

In [12]:
num_sentences = len(X_data)
print("Sliced our corpus into {0} sentences of length {1}".format(num_sentences, sentence_length))

Sliced our corpus into 219628 sentences of length 20


In [13]:
# now we need one-hot encoding
print("Vectorizing X and y...")
X = np.zeros((num_sentences, sentence_length, num_chars), dtype=bool)
y = np.zeros((num_sentences, num_chars), dtype=bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Vectorizing X and y...


In [14]:
# Define our model
print("Let's build model.")
model = Sequential()
model.add(LSTM(32, input_shape=(sentence_length, num_chars), return_sequences=True))
model.add(Bidirectional(LSTM(32, return_sequences=True)))  
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary() 

Let's build model.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 32)            16128     
                                                                 
 bidirectional (Bidirection  (None, 20, 64)            16640     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 20, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 93)                6045      
                                     

Een Long Short-Term Memory (LSTM) laag is een type van Recurrent Neural Network (RNN) laag die speciaal is ontworpen om het "verdwijnende gradiëntprobleem" te overwinnen, een probleem dat optreedt bij het trainen van traditionele RNN's.

LSTM's lossen dit probleem op door een soort "geheugencel" te introduceren die informatie kan opslaan en ophalen over lange perioden. Dit maakt ze bijzonder effectief voor taken waarbij context uit het verleden belangrijk is, zoals taalmodellering of tijdreeksanalyse.

In een LSTM laag, wordt elke neuron vervangen door een LSTM cel. Deze cel heeft drie poorten: een inputpoort, een forget (vergeet) poort, en een outputpoort. Deze poorten bepalen respectievelijk hoeveel van de nieuwe informatie wordt opgeslagen, hoeveel van de oude informatie wordt vergeten, en hoeveel van de huidige celstaat wordt uitgevoerd naar de volgende cel. Deze mechanismen stellen de LSTM in staat om relevante informatie over langere sequenties te behouden.

In [15]:
# Train our model! save the weights after each epoch if it's a new best for loss and save model architecture.
architecture = model.to_json()
with open('model.json', 'w') as model_file:
    model_file.write(architecture)

# Set up checkpoints, and save trained model
file_path="weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks = [checkpoint]



Het slaat de architectuur van het model op in een JSON-bestand. Dit is handig omdat je het model later opnieuw kunt laden met dezelfde architectuur, zelfs als je het script opnieuw uitvoert of het in een andere omgeving uitvoert.

Het stelt een checkpoint in voor het model tijdens het trainingsproces. Een checkpoint is een manier om de voortgang van het model op te slaan tijdens het trainen. In dit geval wordt het model opgeslagen na elk tijdperk (epoch) als het huidige verlies (loss) lager is dan het verlies van alle vorige tijdperken. Dit betekent dat aan het einde van de training, het bestand `weights-{epoch:02d}.hdf5` de gewichten van het model zal bevatten op het punt waar het het laagste verlies had. Dit is handig omdat neurale netwerken vaak over een lange tijd trainen, en als het proces om welke reden dan ook wordt onderbroken, wil je niet helemaal opnieuw beginnen. Met checkpoints kun je verdergaan waar je gebleven was, of het model laden dat het beste presteerde tijdens de training.

In [16]:
# Train het model voor minimaal 20 epochs, En selecteer het dat weights-file die de beste resultaten geeft?
# Is er ook een weights-file waarvanje kunt zeggen dat er overfitting optreedt?

# kijk goed naar de onderstaande regel.

history = model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks)

Epoch 1/20
1716/1716 [==============================] - ETA: 0s - loss: 2.8916
Epoch 1: loss improved from inf to 2.89159, saving model to weights-01.hdf5
1716/1716 [==============================] - 80s 45ms/step - loss: 2.8916
Epoch 2/20
   3/1716 [..............................] - ETA: 1:15 - loss: 2.4886

/Users/jonathanricardo/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1716/1716 [==============================] - ETA: 0s - loss: 2.4217
Epoch 2: loss improved from 2.89159 to 2.42174, saving model to weights-02.hdf5
1716/1716 [==============================] - 104s 60ms/step - loss: 2.4217
Epoch 3/20
1716/1716 [==============================] - ETA: 0s - loss: 2.2302
Epoch 3: loss improved from 2.42174 to 2.23016, saving model to weights-03.hdf5
1716/1716 [==============================] - 115s 67ms/step - loss: 2.2302
Epoch 4/20
1716/1716 [==============================] - ETA: 0s - loss: 2.1091
Epoch 4: loss improved from 2.23016 to 2.10914, saving model to weights-04.hdf5
1716/1716 [==============================] - 121s 70ms/step - loss: 2.1091
Epoch 5/20
1715/1716 [============================>.] - ETA: 0s - loss: 2.0223
Epoch 5: loss improved from 2.10914 to 2.02234, saving model to weights-05.hdf5
1716/1716 [==============================] - 124s 72ms/step - loss: 2.0223
Epoch 6/20
1716/1716 [==============================] - ETA: 0s - loss: 1.9

`callbacks=callbacks`: Callbacks zijn functies die kunnen worden aangeroepen op bepaalde stadia van het trainingsproces, zoals aan het einde van elke epoch. Hier gebruiken we `ModelChekcpoint` die we eerder gemaakt hebben, die het model opslaat na elke epoch als het huidige verlies lager is dan het verlies van alle vorige epochs.

